# 19_NestJS_Mapped_Types_and_PartialType_Update_User_DTO_Patch_Request

### 1. What are Mapped Types in NestJS?

* **Mapped Types** are utilities provided by NestJS to help you **reuse and transform DTOs** instead of rewriting them.
* They are especially useful when you need:

  * A similar DTO with **slightly different rules** (e.g., fields optional for PATCH).
  * To **avoid duplication** and follow the **DRY principle**.
* Mapped types are related to DTOs and help you organize them in a clean, scalable way.

Common mapped-type helpers (from `@nestjs/swagger`):

* `PartialType()` – makes **all properties optional**.
* `PickType()` – picks a **subset** of properties.
* `OmitType()` – omits specific properties.
* `IntersectionType()` – combines multiple DTOs.

In this lecture, the focus is on **`PartialType`**.

---

### 2. The Use Case: `PATCH /users` – Partial Update

We want to add a new controller method in `UsersController` to handle **partial updates** of a user:

```ts
@Patch()
updateUser(@Body() body: any) {
  // will update the user partially
}
```

Key ideas about a **PATCH** request:

* PATCH is used to **partially update** a resource.
* The request body **does not contain the full user object**.
* It only includes the properties that need to be updated.

**Example patch request body (Postman):**

```json
{
  "email": "john1@gmail.com",
  "isMarried": true
}
```

* We are **not** sending `id`, `name`, or `gender` because we don’t want to change them.

So, the DTO for PATCH **cannot be identical** to the DTO used for `CREATE`, where most fields are required.

---

### 3. Why `CreateUserDto` is NOT Suitable for Patch

We already have this DTO for creating a user:

```ts
export class CreateUserDto {
  @IsNumber()
  id: number;

  @IsString({ message: 'Name should be a string value' })
  @IsNotEmpty()
  @MinLength(3, { message: 'Name is too short. Minimum length is 3 characters' })
  name: string;

  @IsString()
  @IsOptional()
  gender?: string;

  @IsEmail()
  email: string;

  @IsBoolean()
  isMarried: boolean;
}
```

* For **create**, most fields (`id`, `name`, `email`, `isMarried`) are **required**.
* If we use `CreateUserDto` for PATCH:

```ts
@Patch()
updateUser(@Body() body: CreateUserDto) {}
```

And we send only:

```json
{
  "email": "john1@gmail.com",
  "isMarried": true
}
```

Then validation fails because:

* `name` is missing → `name should not be empty`.
* `name` is required and has `@IsNotEmpty()` and `@MinLength()`.

So, `CreateUserDto` **forces all required fields**, which is not what we want for **partial update**.

---

### 4. First Attempt (Naive): Creating a Separate `UpdateUserDto`

We could create a separate DTO file:

`update-user.dto.ts`

```ts
import { IsBoolean, IsEmail, IsNotEmpty, IsNumber, IsOptional, IsString, MinLength } from 'class-validator';

export class UpdateUserDto {
  @IsNumber()
  @IsOptional()
  id?: number;

  @IsString({ message: 'Name should be a string value' })
  @IsNotEmpty()
  @MinLength(3, { message: 'Name is too short. Minimum length is 3 characters' })
  @IsOptional()
  name?: string;

  @IsString()
  @IsOptional()
  gender?: string;

  @IsEmail()
  @IsOptional()
  email?: string;

  @IsBoolean()
  @IsOptional()
  isMarried?: boolean;
}
```

Usage in controller:

```ts
@Patch()
updateUser(@Body() body: UpdateUserDto) {
  console.log(body);
}
```

* This works: all properties are **optional**, so we can send **only `email` and `isMarried`**.
* But we have a **big drawback**:

  * We **duplicated all properties and validators** from `CreateUserDto`.
  * If later we add a new field (e.g. `phone`) to `CreateUserDto`, we must **remember to update** `UpdateUserDto` too.
  * This **violates DRY (Don’t Repeat Yourself)**.

We need a better way → **Mapped Types**.

---

### 5. Installing `@nestjs/swagger`

Mapped types are provided by the `@nestjs/swagger` package.

Install it via terminal:

```bash
npm install @nestjs/swagger
```

Check `package.json` to confirm:

```json
"dependencies": {
  "@nestjs/swagger": "^..."
}
```

Now we can import `PartialType`, `PickType`, etc.

---

### 6. Refactoring `UpdateUserDto` with `PartialType`

We clean up `update-user.dto.ts` and refactor it using mapped types:

```ts
// update-user.dto.ts
import { PartialType } from '@nestjs/swagger';
import { CreateUserDto } from './create-user.dto';

export class UpdateUserDto extends PartialType(CreateUserDto) {}
```

What this line does:

```ts
export class UpdateUserDto extends PartialType(CreateUserDto) {}
```

**`PartialType(CreateUserDto)` will:**

1. Take **all properties** from `CreateUserDto` (`id`, `name`, `gender`, `email`, `isMarried`).
2. Copy **all validation decorators** from `CreateUserDto` (e.g. `@IsEmail()`, `@IsBoolean()`, `@MinLength()`, etc.).
3. Make **all properties optional** in `UpdateUserDto`.

So `UpdateUserDto` is equivalent to the big manually-written `UpdateUserDto` earlier, but with **one single line** and **no duplication**.

Benefits:

* Follows **DRY principle**.
* If you later add a new property to `CreateUserDto`, it **automatically appears** in `UpdateUserDto`.
* You don’t need to repeat validators.

---

### 7. Using `UpdateUserDto` in the Controller

Updated `UsersController` (relevant part):

```ts
import { Controller, Get, Param, ParseIntPipe, Post, Query, DefaultValuePipe, Patch, Body } from '@nestjs/common';
import { UsersService } from './users.service';
import { CreateUserDto } from './dtos/create-user.dto';
import { UpdateUserDto } from './dtos/update-user.dto';

@Controller('users')
export class UsersController {
  // ... other endpoints

  @Patch()
  updateUser(@Body() user: UpdateUserDto) {
    console.log(user); // partial user object
    return 'User updated successfully';
  }
}
```

Behavior:

* If the request body contains **only** `email` and `isMarried`, it passes validation since those fields exist in `UpdateUserDto` and are optional.
* If an invalid value is passed (e.g., non-email string for `email`), the **same validators** from `CreateUserDto` still apply.

Example valid PATCH body:

```json
{
  "email": "john1@gmail.com",
  "isMarried": true
}
```

Example invalid PATCH body:

```json
{
  "email": "not-an-email",
  "isMarried": "yes"
}
```

* Will trigger validation errors for `email` and `isMarried`.

---

### 8. How Mapped Types Help in Real Projects

Mapped types like `PartialType` are incredibly useful when:

* You have a **Create** DTO (all required), and you need an **Update** DTO (all optional).
* You want to avoid **copy–paste** across multiple DTOs.
* You want your DTOs to **stay in sync** automatically.

Typical patterns:

* **Create DTO** – required properties:

  * `CreateUserDto`, `CreatePostDto`, `CreateProductDto`, etc.
* **Update DTO** – all optional (PATCH):

  * `UpdateUserDto extends PartialType(CreateUserDto)`
  * `UpdatePostDto extends PartialType(CreatePostDto)`

You can also combine `PartialType` with others (e.g., `PickType`, `OmitType`) for more advanced scenarios.

---

### 9. Summary

* **Mapped Types** in NestJS help you reuse DTOs without repetition.
* `PartialType(CreateUserDto)` creates a new DTO where:

  * All properties of `CreateUserDto` are copied.
  * All validators are preserved.
  * All fields become **optional**.
* This is perfect for **PATCH** endpoints where the client sends a **partial object**.
* You avoid duplication, follow **DRY**, and keep validation consistent across your app.

> If you later add or modify fields in `CreateUserDto`, `UpdateUserDto` (using `PartialType`) automatically stays up-to-date.
